 # 実験1 (パーサ（構文解析器）の理解)
 - パーサ（構文解析器）を小さな部品を組み合わせて系統的に作成する方法を学ぶ．
 - 実例としてコンマ区切りデータ(CSV)を解析する．

 ライブラリの読み込み（最初に必ずおこなう）

In [ ]:
from toyparsing import *

 ## 10進整数を切り出すパーサ
 10進整数のパターンを正規表現で表し`pat`(`pattern`)関数を呼び出すと10進整数を切り出すパーサが作られる

In [ ]:
num = pat("[0-9]+")

 つくられた`num`パーサに入力文字列を与えて呼び出すと，先頭から10進整数の部分を切り出して返す．そのとき，やり残りの文字列とペア（対）にして返す．

In [ ]:
num("123abc")

 切り出しに失敗することもある．以下の例は先頭に10進整数のパターンが無いので失敗．失敗すると`None`（Pythonで未定義を表す特別な値）を返す．Jupyterノートブックでは`None`は表示されないが，`print`関数を使えば確認できる．

In [ ]:
print(num("abc123"))

 ## パーサの概念図
 パーサは文字列を入力とする関数（函数）だが，概念的には出力が2つあると考えると理解しやすい．つまり処理結果を返す出口の他に，やり残した文字列を返す出口がある．

 <img src="img/num_parser.png" width=50%>

 ## パーサの直列接続（流れ作業）
 パーサを連結して流れ作業でより複雑な処理をすることができる．以下の例は`num`パーサを3つ直列につないで流れ作業で3つの連続する（空白で区切られた）10進整数を切り出すパーサ`num3`をつくっている．
 デコレータ`@parser_do`は，後続の関数定義がパーサの定義として以下のように特別に扱われるように指示している．
 1. パーサの定義は必ず１引数で仮引数名はなんでもいいが，普通は`run`という名前にする．
 2. `n1`, `n2`, `n3`には切り出した10進整数文字列が入る（やり残し文字列とのペアではない！）やり残し文字列を後続の`num`パーサに渡す仕事はコードには現れないが裏で自動的におこなわれる．
 3. 途中で失敗したときには，残りの処理はスキップされ**`num3`全体が失敗**して`None`を返す．このとき，num3の処理中に読んだ入力文字はどれも読まなかったことになりnum3開始時点の状態にまで戻される．
 4. 最後に`n1`,`n2`,`n3`の値を要素とするリスト（動的配列）を返している（コードには現れないが自動的にやり残し文字列とペアにしてから返される）．

 以下のようなイメージをもつと理解しやすい．

 <img src="img/serial.png" width=100%>

In [ ]:
@parser_do
def num3(run):
    n1 = run(num)
    n2 = run(num)
    n3 = run(num)
    return [n1,n2,n3]

In [ ]:
num3("12 3 456 78")

In [ ]:
print(num3("12 abc 345"))

 直列接続（流れ作業）は頻出なので，これを短く書ける`&`演算子が用意されている．
 `&`は左結合なのでパーサの結果は左から順にふたつずつまとめて返される．

In [ ]:
num2 = num & num
num2("12 3 456 78")

In [ ]:
num4 = num & num & num & num 
num4("12 3 456 78")

 以下の例（誤例）では結果をリストにまとめて返す代わりに和を求めて返すつもりだが，思ったように動かない（`n1`,`n2`,`n3`は整数ではなく整数を表す文字列なので，たし算の代わりに文字列の連結が実行されてしまっている）．

In [ ]:
@parser_do
def num3_sum(run):
    n1 = run(num)
    n2 = run(num) 
    n3 = run(num)
    return n1 + n2 + n3

In [ ]:
num3_sum("12 3 456 78")

 `num`パーサを呼び出す度に整数に変換するよりは，最初から整数を返すように`num`パーサの定義を改良するほうが合理的．
 以下のセルを実行した後，上の例を再度実行すると意図通り正しく総和が求められることが分かる．

In [ ]:
@parser_do
def num(run):
    return int(run(pat("[0-9]+")))

 ## 失敗の制御
 前述のようにパーサの定義においては，部分の解析が失敗すると自動的に全体を失敗させるのがデフォルト（既定）の動作だが，
 `run`関数にオプション`nullable=True`を与えるとこれを無効にして失敗時の挙動を手動で制御できるようになる．
 `run(p, nullable=True)`でパーサ`p`の実行が失敗すると
 1. `run(p, nullable=True)`は`None`を返し
 2. パーサ`p`の処理中に読んだ入力文字列は読まなかったことになり
 3. パーサ`p`はNoneを返す．

 残りの処理をどうするかは返された値が`None`かどうかをみてプログラマが自分で制御することになる．
 以下はnum3とほとんど同じだが、「nullable=True」が付加されている点だけが異なる．
 実行して結果を観察せよ．

In [ ]:
@parser_do
def num3_nullable(run):
    n1 = run(num, nullable=True)
    n2 = run(num, nullable=True)
    n3 = run(num, nullable=True)
    return [n1,n2,n3]

In [ ]:
num3_nullable("a 2 3 4")

In [ ]:
num3_nullable("1 a 3 4")

In [ ]:
num3_nullable("1 2 a 4")

In [ ]:
num3_nullable("1 2 3 4")

 この挙動をもとに、高々3つまでの数を認識するパーサは以下のように実現できる．

In [ ]:
@parser_do
def num_at_most3(run):
    n1 = run(num, nullable=True)
    if n1 is None: return []
    n2 = run(num, nullable=True)
    if n2 is None: return [n1]
    n3 = run(num, nullable=True)
    if n3 is None: return [n1,n2]
    return [n1,n2,n3]

In [ ]:
num_at_most3("a 2 3 4")

In [ ]:
num_at_most3("1 a 3 4")

In [ ]:
num_at_most3("1 2 a 4")

In [ ]:
num_at_most3("1 2 3 4")

 どんな入力に対しても常に失敗するパーサ`fail`が用意されている．
 強制的に失敗させたいときには、これをrunすればよい．

In [ ]:
print(fail("123"))

 ## パーサの並列接続（バックアップ，下支え）
 あるパーサを別のパーサのバックアップ（下支え）として並列して使うこともある．以下は英小文字の並びを切り出す`alpha`パーサを`num`パーサのバックアップとして用いる例．
 `num_or_alpha`は，最初に`num`を用いて試行し，切り出しに失敗したときに限り`alpha`を用いる．

In [ ]:
alpha = pat("[a-z]+")

@parser_do
def num_or_alpha(run):
    n = run(num, nullable=True)  # [問]`nullable=True`を取り除いて実行するとどうなるか？
    if n is None:
        return run(alpha)
    return n

In [ ]:
num_or_alpha("123abc")  # `num`が使われて123を切り出す（`alpha`は未使用）

In [ ]:
num_or_alpha("abc123")  # `num`は失敗するので下支えの`alpha`が使われてabcが切り出される

 以下のようなイメージをもつと理解しやすい．

 <img src="img/parallel.png" width=90%>

 並列接続（バックアップ）もよく用いられるので短く書けるよう演算子`|`が用意されている．以下の例は上の`num_or_alpha`と等価．

In [ ]:
num_or_alpha_alt = num | alpha

 ## パーサの反復結合
 パーサの直列接続（流れ作業）を反復しておこなうこともある．以下の例は(空白で区切られた)10進整数の並びを可能な限り長く切り出すパーサの例．10進整数の切り出しに失敗すればwhileループから抜け，その直前までの結果をリストにまとめて返している．
 `num`の呼び出しはいずれは（10進整数でない文字列か文字列全体の最後にたどり着いて）必ず失敗するので，このループはいずれは停止する．

In [ ]:
@parser_do
def nums(run):
    ns = []
    while True:
        n = run(num, nullable=True) # [問]`nullable=True`を取り除いて実行するとどうなるか？
        if n is None: break
        ns += [n]
    return ns

In [ ]:
nums("12 3 45 6 abc 78 9")

 数をひとつも切り出せないこともありうる．このときは，上記の定義によれば，空のリスト`[]`を返すことになる．よって`nums`パーサはどんな入力に対しても失敗することがない．

In [ ]:
nums("abc 12 3 45")

 上記のnumsのようにwhile文で反復するプログラムを書くときは無限ループにならないように注意(課題1(1)参照)．

 ## その他のライブラリ関数
 上記以外にもパーサをつくったり組み合わせるのに便利な様々な関数が用意されているが，その中で本実験で用いるのは以下の4つだけである．
 (他の関数の説明は省略しますが使用は禁止しませんので，ライブラリのソースコード`toyparsing.py`を参照して使って構いません．)

 ### `word` （略記: `w`）
 `pat` (`pattern`)と似ているが正規表現特有のメタ文字(`*`,`+`,`|`など)を解釈しない（例：`word("[0-9]+")` は`pat(”\[0-9\]\+”)`と同じ意味になる．つまり，`[`, `0`, `-`, `9`, `]`, `+` の6文字からなる文字列を切り出すパーサを作って返す．

In [ ]:
p = w("[0-9]+")
p("[0-9]+")

 ### `>>`
 `p >> q`は`p & q`と同じく直列接続だが，先行する`p`の結果が捨てられるところだけが違う．`&`より先に結合する．

In [ ]:
p = alpha >> num
p("abc 123 45")

 ### `<<`
 `p << q`は`p & q`と同じく直列接続だが，後続の`q`の結果が捨てられるところだけが違う．`&`より先に結合する．

In [ ]:
p = alpha << num
p("abc 123 45")

 ### `>`
 `p > f`は`p`の切り出した出力に関数`f`を作用させるようなパーサをつくって返す．つまり以下のパーサ`f_of_p`と等価．
 ``` python
 @parser_do
 def f_of_p(run):
     return f(run(p))
 ```

 例えば上ででてきた整数に変換してから返す`num`の定義は以下のように短く書ける．

In [ ]:
num = pat("[0-9]+") > int

 ## コンマ区切り形式（CSV）データの処理
 以下はコンマで区切られた10進整数の並びをできるだけ長く切り出す．

In [ ]:
comma = w(",")  

In [ ]:
@parser_do
def csv(run):
    a = [run(num)]
    while True:
        r = run(comma & num, nullable=True)
        if r is None: break
        a += r
    return a    

In [ ]:
csv("12,3,456,7,a,89")

 コンマで区切られた10進整数の総和を求める．

In [ ]:
@parser_do
def csv_sum(run):
    a = run(num)
    while True:
        b = run(comma >> num, nullable=True)
        if b is None: break
        a += b
    return a

In [ ]:
csv_sum("1,2,3,4,5,6,7,8,9,10")

 ### [課題1]
 (1) 以下のパーサに入力を与えて呼び出すとどうなるか？
 
 (A) 必ず停止して（Noneも含む）何らかの値を返す
 
 (B) どんな入力に対しても無限ループになる
 
 (C) 入力次第で停止して何らかの値を返すこともあれば、無限ループになることもある
 
 正しいと考える挙動を(A)〜(C)から選び、その理由をすじみち立てて説明せよ．

In [ ]:
@parser_do
def double_nums(run):
    ns = []
    while True:
        n = run(nums, nullable=True) 
        if n is None: break
        ns += [n]
    return ns 

 (2) 以下の`csv_sum_alt1`と`csv_sum_alt2`は上記の`csv_sum`の別解だろうか？ つまり`csv_sum_alt`と`csv_sum`はどんな入力に対しても同じふるまいをするのだろうか？
 同じならそのことをすじみち立てて説明せよ．異なる場合はそのことを示す最短の（つまり文字数の少ない）入力文字列の例（反例）を提示し、それが最短の反例になっている理由を説明せよ．

In [ ]:
@parser_do
def csv_sum_alt1(run):
    a = run(num)
    while True:
        if run(comma, nullable=True) is None: break
        a += run(num)
    return a 

In [ ]:
@parser_do
def csv_sum_alt2(run):
    a = run(num)
    while True:
        if run(comma, nullable=True) is None: break
        n = run(num, nullable=True)
        if n is None: break
        a += n
    return a